In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy import stats,sparse
from sklearn.base import TransformerMixin
from datetime import datetime as dt
from math import isnan
from numpy import ma
import pickle as pickle
import xgboost as xgb
import time
from pandas import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 

import json
from sklearn.metrics import roc_curve, auc
from re import sub
from collections import defaultdict
import scipy as sp

from sklearn.cross_validation import StratifiedKFold,cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
from sklearn import metrics

In [21]:
FeaturesSelected=np.load('./mostrecentinput/XGBoost_Oct172015212459_AUC_0.764813590574FeatNum199.p')
X=np.load('./mostrecentinput/nxtrain_standard_original0.npy')
X1= np.load('./mostrecentinput/nxtrain_standard_derived0.npy')
X2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_train2.dat")
X3=pd.read_pickle("./mostrecentinput/time_series_original_standard_train2.dat")
X4=pd.read_pickle("./mostrecentinput/cat_numeric_th60_standard_train2.dat")
X5=pd.read_pickle("./mostrecentinput/cat_le_train2.dat") 
X=np.hstack((X,X1,X2,X3,X4,X5))
y=pickle.load(open("./mostrecentinput/ytrain2.dat","rb"))
xtrain=X[:,FeaturesSelected]
del X,X1,X2,X3,X4,X5


In [23]:
X_test=np.load('./mostrecentinput/nxtest_standard_original0.npy')
X_test1= np.load('./mostrecentinput/nxtest_standard_derived0.npy')
X_test2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_test2.dat")
X_test3=pd.read_pickle("./mostrecentinput/time_series_original_standard_test2.dat")
X_test4=pd.read_pickle("./mostrecentinput/cat_numeric_th60_standard_test2.dat")
X_test5=pd.read_pickle("./mostrecentinput/cat_le_test2.dat") 
X_test=np.hstack((X_test,X_test1,X_test2,X_test3,X_test4,X_test5))

xtest=X_test[:,FeaturesSelected]
del X_test1,X_test2,X_test3,X_test4,X_test5
TestID=pd.read_pickle("./mostrecentinput/TestID.p")

In [17]:
trainpreds=pd.read_pickle('trainpreds_withoutsgd_0p784555_wl_.pickle')
testpreds=pd.read_pickle('testpreds_withoutsgd_0p784555_wl_.pickle')
xtrain=np.hstack((xtrain,trainpreds))
xtest=np.hstack((xtest,testpreds))

In [ ]:
for randomstatenumber in [8421,45212,43454,65234,4546]:
    KFoldNumber=3
    num_boost=1000
    kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state=randomstatenumber)
    ypreds_test_wl=pd.DataFrame()
    ypreds_train_wl=pd.DataFrame()
    ypreds_cvindex_wl=pd.DataFrame()
    AUCValues=np.zeros(KFoldNumber)
    kfoldnum=0
    train_predicted=np.zeros(y.shape)
    params = { 'max_depth': 12, 
            'eta': 0.01, 
            'objective':'binary:logistic', 
            'subsample': 1,
            'colsample_bytree': 0.4,
            'eval_metric': 'auc',
            'min_child_weight': 3
           }

    for train,cv in kfold:
        kfoldnum=kfoldnum+1
        cv_index=np.zeros(y.shape)

        dtrain = xgb.DMatrix(xtrain[train,:], label=y[train])
        clf = xgb.train(params=params,dtrain=dtrain, num_boost_round=num_boost)    

        dfulltrain = xgb.DMatrix(xtrain, label=y)
        preds_fulltrain = clf.predict(dfulltrain)    
        dfulltest = xgb.DMatrix(xtest, label=np.zeros(xtest.shape[0]))
        preds_fulltest = clf.predict(dfulltest)
        preds_cv=preds_fulltrain[cv]
        cv_index[cv]=1

        AUC1=metrics.roc_auc_score(y[cv], preds_cv)

        ypreds_test_wl['CV'+ str(kfoldnum)] = preds_fulltest
        ypreds_train_wl['CV'+ str(kfoldnum)] = preds_fulltrain
        ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
        AUCValues[kfoldnum-1]=float(AUC1)
        print(AUCValues)

    trainPredict=(ypreds_train_wl*ypreds_cvindex_wl).sum(axis=1)
    testPredictMean=ypreds_test_wl.mean(axis=1)
    AUC1=metrics.roc_auc_score(y, trainPredict)
    print(AUC1)

    TestPredict=pd.DataFrame()
    TestPredict['ID']=TestID
    TestPredict['Probability']=testPredictMean


    str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
    filename0='FinalModelPrediction_xgb_Meta'+str1+'_niter2000_AUC_0p'+ str(int(AUC1.mean()*1e6))
    filename=filename0+ '.p'
    pickle.dump((trainPredict,TestPredict,ypreds_train_wl,ypreds_test_wl,ypreds_cvindex_wl,AUCValues), 
                open(filename,'wb'), protocol =2) 

    pn='C:/Users/liw5/Dropbox/Bio_Physics_JailBreak/Kaggle/Final Prediction/'
    save_train_name=pn + filename0 +'_wl_ypredtrain.pkl'
    save_test_name=pn + filename0 +'_wl_ypredtest.pkl'

    pickle.dump(trainPredict,open(save_train_name,'wb'),protocol=2)
    pickle.dump(testPredictMean,open(save_test_name,'wb'),protocol=2)

    print('Model prediction done!')